Import Packages

In [1]:
import numpy as np
import pandas as pd
from netCDF4 import Dataset
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, Point, MultiPolygon, shape
import fiona
import geopandas as gpd

Open and read-in Files

In [2]:
#Open Plant Dataset
county_fips = pd.read_csv('county_fips.csv')
fips = np.array(county_fips).tolist()

#Import NERC Shapefiles
nerc = gpd.read_file('NERC_regions/NercRegions_201907.shp')

#Import EGU Datafile
#Remove non-contigous states
egu = pd.read_csv('eGRID_plnt16.csv')
egu.drop(egu[egu['Plant state abbreviation'] == 'AK'].index, inplace = True)
egu.drop(egu[egu['Plant state abbreviation'] == 'HI'].index, inplace = True)
#Remove header row
egu = egu[1:]

#Import GENERATOR Egu file
egen = pd.read_csv('egrid2016_gen16.csv')
egen.drop(egen[egen['Plant state abbreviation'] == 'AK'].index, inplace = True)
egen.drop(egen[egen['Plant state abbreviation'] == 'HI'].index, inplace = True)
#Remove header row
egen = egen[1:]

#Import County Shapefiles
#We need county shapefiles to determine what counties lie within which NERC region
counties = gpd.read_file('CONUS_shp/US_county_2018.shp')
counties = counties.to_crs("EPSG:4326")

#California VMT file
caVMT = pd.read_csv('california_vmt.csv')
caVMT = np.array(caVMT).tolist()

#Smoke File import
smoke_file = Dataset('inln_mole_ptegu_20160101_CONUS4K_d02_cmaq_cb6_2016ff_16j.ncf')
smoke_egu = Dataset('stack_groups_ptegu_CONUS4K_d02_2016ff_16j.ncf')
vmt_data = pd.read_csv('VMT_tail+29.csv')
vmt_full = np.array(vmt_data).tolist()
vmt_full.insert(0, ['US', 32003, 'nan', 'nan', 'nan', 2201110200, 'nan', 'nan', 'VMT', 220929.239490000007, 2016, 20180727, '2016_beta_state_via_CSRA', 5788.78808279999976, 5235.13920520000011, 12886.3604529999993, 22248.9007889999993, 26376.5204339999982, 28037.6888719999988, 29447.2156009999999, 29799.5989300000001, 25017.5850839999985, 20990.486488999999, 9765.29365160000089, 5335.66190009999991, 'c32003y2014_20170911  Results assume the CDB  has the new  HPMS system. No conversion of HPMS.'])
vmt_full.insert(0, ['country_cd', 'region_cd', 'tribal_code', 'census_tract_cd', 'shape_id', 'scc', 'act_parm_type_cd', 'act_parm_uofmsr', 'activity_type', 'ann_parm_value', 'calc_year', 'date_updated', 'data_set_id', 'jan_value', 'feb_value', 'mar_value', 'apr_value', 'may_value', 'jun_value', 'jul_value', 'aug_value', 'sep_value', 'oct_value', 'nov_value', 'dec_value', 'comment'])

/home/mav5149/.conda/envs/mvenviro/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (0,3,5,7,14,15,17,18,19,20,24,26,31,47,48,49,51,52,55,56,57,58,60,61,62,64,65,84,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/mav5149/.conda/envs/mvenviro/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (12,25) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
#Determine proportion of renewable electricity generated by each NERC region
egen = np.array(egen).tolist()
egu = np.array(egu).tolist()

mro = [i for i in egu if i[10] == 'MRO']
npcc = [i for i in egu if i[10] == 'NPCC']
rfc = [i for i in egu if i[10] == 'RFC']
serc = [i for i in egu if i[10] == 'SERC' or i[10] == 'FRCC']
spp = [i for i in egu if i[10] == 'SPP']
tre = [i for i in egu if i[10] == 'TRE']
wecc = [i for i in egu if i[10] == 'WECC']

nercs = [mro, npcc, rfc, serc, spp, tre, wecc]

fracs_by_nerc = []
for i in nercs:
    renew_frac = []
    non_renew_frac = []
    for n in i:
        if n[22] == 'SOLAR' or n[22] == 'WIND' or n[22] == 'HYDRO' or n[22] == 'GEOTHERMAL':
            renew_frac.append(n[37])
        else:
            non_renew_frac.append(n[37])
    r = []
    nr = []
    for i in renew_frac:
        if type(i) == float:
            pass
        elif ',' in i:
            r.append(int(i.replace(',', '')))
        elif i != 'nan':
            r.append(int(i))
    for i in non_renew_frac:
        if type(i) == float:
            pass
        elif ',' in i:
            nr.append(int(i.replace(',', '')))
        elif ',' not in i and i != 'nan':
            nr.append(int(i))
    r_percent = (sum(nr) / ((sum(r)) + sum(nr)))
    fracs_by_nerc.append(r_percent)

In [10]:
#Remove any/all powerplants and generators who's ORIS code is not in SMOKE's inventory, per ptegu_list_2016.
#Add back in nuclear EGU's

ptegu_list = pd.read_csv('ptegu_list_2016.csv')
ptegu_list = np.array(ptegu_list).tolist()
discrete_ptegus = []
for i in ptegu_list:
    if str(i[4]) not in discrete_ptegus:
        discrete_ptegus.append(str(i[4]))
nuc_oris = []
for i in egu:
    if i[22] == 'NUCLEAR':
        discrete_ptegus.append(str(i[3]))
        nuc_oris.append(str(i[3]))

new_egu = []
for i in egu:
    if str(i[3]) in discrete_ptegus:
        new_egu.append(i)
        
new_egen = []
for i in egen:
    if str(i[3]) in discrete_ptegus:
        new_egen.append(i)
        
egu = new_egu
egen = new_egen

del new_egu
del new_egen

In [12]:
#Add powerplant age (compiled by averaging all generators of a powerplant) for age weight later on.
#If no age, use 1982 average since that is current egu age per EIA.

tally = []
for i in egu:
    m = []
    m.append(str(i[3]))
    tally.append(m)

numbers = []
for i in egen:
    l = []
    k = i[3]
    m = i[14]
    l.append(k)
    l.append(m)
    numbers.append(l)


for i in tally:
    m = []
    for n in numbers:
        if i[0] == n[0]:
            m.append(n[1])
    int_list = []
    for q in m:
        if len(str(q)) == 4:
            int_list.append(int(q))
    if len(int_list) == 0:
        i.append(1982)
    else:
        average = sum(int_list)/len(int_list)
        i.append(average)

Assign Variables

In [25]:
#Electric Vehicle Adoption Scenario
ev_penetration = .3 # Depending on experimental scenario
grid_loss = .051 #5.1% Energy is lost in transmission through the energy grid per 2019 EPA

In [31]:
#Vehicle efficiencies MWh/mile
eff_41 = .001613 # Ebusco 2.2 
eff_42 = .002006 # Proterra ZX5
eff_43 = .0013548 # Lion Electric (155-mile range)
eff_51 = .0019764 # Lion8 Refuse Truck
eff_52 = .002 # Tesla Semi
eff_53 = .002 # Tesla Semi
eff_54 = .0004337 # WOF Iridium
eff_61 = .002 # Tesla Semi
eff_62 = .002 # Tesla Semi

In [17]:
#Pull Out relevant SCC Codes from VMT file
all_scc = []
for i in vmt_full:
    if str(i[5]) not in all_scc:
        all_scc.append(str(i[5]))
#Turn string SCCs into integers and rename twice lol.
ld_scc = all_scc[1:]
for i in range(len(ld_scc)):
    ld_scc[i] = int(ld_scc[i])
scc_codes = ld_scc

#Add california counties to VMT_full
for i in caVMT:
    vmt_full.append(i)

#This does nothing. Keep as relic of my stupidity. xoxo -MAV
vmt_ldv = []
for i in vmt_full:
    for n in scc_codes:
        if i[5] == n:
            vmt_ldv.append(i)
            
            
vmt_ldv.insert(0, ['country_cd', 'region_cd', 'tribal_code', 'census_tract_cd', 'shape_id', 'scc', 'act_parm_type_cd', 'act_parm_uofmsr', 'activity_type', 'ann_parm_value', 'calc_year', 'date_updated', 'data_set_id', 'jan_value', 'feb_value', 'mar_value', 'apr_value', 'may_value', 'jun_value', 'jul_value', 'aug_value', 'sep_value', 'oct_value', 'nov_value', 'dec_value', 'comment'])
#Delete large datasets to clear up memory
del caVMT

In [32]:
#Combine All SCC's to create 1 VMT total per county
county_ids_long = []
for i in vmt_ldv:
    if i[1] not in county_ids_long:
        county_ids_long.append(i[1])
county_ids_long = county_ids_long[1:]
county_ids = []
for i in county_ids_long:
    county_list = []
    county_list.append(i)
    county_ids.append(county_list)
for i in county_ids:
    for n in vmt_ldv:
        if n[1] == i[0]:
            if str(n[5])[4:6] == '41':
                i.append((n[9]*ev_penetration*eff_41)/(1 - grid_loss))
            elif str(n[5])[4:6] == '42':
                i.append((n[9]*ev_penetration*eff_42)/(1 - grid_loss))
            elif str(n[5])[4:6] == '43':
                i.append((n[9]*ev_penetration*eff_43)/(1 - grid_loss))   
            elif str(n[5])[4:6] == '51':
                i.append((n[9]*ev_penetration*eff_51)/(1 - grid_loss))
            elif str(n[5])[4:6] == '52' or str(n[5])[4:6] == '53' or str(n[5])[4:6] == '61' or str(n[5])[4:6] == '62':
                i.append((n[9]*ev_penetration*eff_52)/(1 - grid_loss))
            elif str(n[5])[4:6] == '54':
                i.append((n[9]*ev_penetration*eff_54)/(1 - grid_loss))

county_vmt = []
for i in county_ids:
    county_data = []
    county_data.append(i[0])
    total_cvmt = sum(i[1:])
    county_data.append(total_cvmt)
    county_vmt.append(county_data)
    
#Delete large datasets to clear up memory
#del county_ids_long
#del county_ids
#del county_list
#del county_data

In [36]:
#Relate FIPS code back to Region_ID and state of county in vmt file
for i in county_vmt:
    for n in fips:
        if i[0] == n[0]:
            i.insert(1, n[2])
            i.insert(1, n[1])
#Add Header for clarification
#county_vmt.insert(0, ['FIPS Code', 'County Name', 'County State', 'Total non-E VMT', "EV-Converted VMT", 'Added mWh'])
point = Point(-102.6216, 43.2347)
county_vmt[1763].insert(1, 'Shannon County')
county_vmt[1763].insert(2, 'South Dakota')
#county_vmt[1764].insert(6, point)

In [38]:
egu_nerc = []
for i in egu:
    single_egu = []
    if i[10] == 'MRO':
        single_egu.append([i[3], i[17], i[18],1,0,0,0,0,0,0])
    elif i[10] == 'NPCC':
        single_egu.append([i[3], i[17], i[18],0,1,0,0,0,0,0])
    elif i[10] == 'RFC':
        single_egu.append([i[3], i[17], i[18],0,0,1,0,0,0,0])
    elif i[10] == 'SERC': 
        single_egu.append([i[3], i[17], i[18],0,0,0,1,0,0,0])
    elif i[10] == 'FRCC':
        single_egu.append([i[3], i[17], i[18],0,0,0,1,0,0,0])
    elif i[10] == 'SPP':
        single_egu.append([i[3], i[17], i[18],0,0,0,0,1,0,0])
    elif i[10] == 'TRE':
        single_egu.append([i[3], i[17], i[18],0,0,0,0,0,1,0])
    elif i[10] == 'WECC':
        single_egu.append([i[3], i[17], i[18],0,0,0,0,0,0,1])
    else:
        single_egu.append([i[3], i[17], i[18],0,0,0,0,0,0,0])
    egu_nerc.append(single_egu)
nerccy = []
for i in egu_nerc:
    nerccy.append(i[0])
egu_nerc = nerccy
print(len(egu_nerc))
egu_nerc = pd.DataFrame(egu_nerc)
egu_nerc.columns = ['PFC', 'Lat', 'Lon', 'MRO', 'NPCC', 'RFC', 'SERC', 'SPP', 'TRE', 'WECC',]
nerc_egu = egu_nerc

1910


In [40]:
counties = counties.to_crs("EPSG:4326")
county_centroids = [(counties['geometry'][i]).centroid for i in range(len(counties['geometry']))]

nerc = gpd.read_file('NERC_regions/NercRegions_201907.shp')

nerc_counties = []
for county in county_centroids:
    county_list = []
    for poly in nerc['geometry']:
        if poly.contains(county):
            county_list.append(1)
        else:
            county_list.append(0)
    nerc_counties.append(county_list)

m = np.array(counties).tolist()
for i in range(len(nerc_counties)):
    nerc_counties[i].insert(0, m[i][6])
    nerc_counties[i].insert(0, m[i][4])
    
nerc_counties = pd.DataFrame(nerc_counties)
nerc_counties.columns = ['FIPS', 'County', 'MRO', 'NPCC', 'RFC', 'SERC', 'SPP', 'TRE', 'WECC', '-']

In [44]:
#Import coded and csv'ed nerc_county weights file.
nerc_counties = np.array(nerc_counties).tolist()
for i in nerc_counties:
    if len(str(i[0])) == 4:
        new_fips = ('0' + str(i[0]))
        i[0] = new_fips

#Now let's delete the last column since it's got nothing
for i in nerc_counties:
    del i[-1]

for i in nerc_counties:
    string_vers = str(i[0])
    i[0] = string_vers

#Fixing Counties that are in between Nerc regions:

#05005-05147 Counties in Arkansas are being manually rerouted to SERC
#19007-19185 Counties in Iowa are being manually rerouted to MRO
#29003-29227 Counties in Missouri are being manually rerouted to SERC
#40001-40147 Counties in Oklahoma are being manually rerouted to SPP
#11001 County is Washington DC and goes in RFC
#23023 is Sagadahoc County and goes in Maine in NPCC
#26083-26115 are in Michigan and go in RFC
#34017 is Hudson County in NJ and belongs in RFC
#37133 and 37137 are counties in NC and belong in SERC
#44005 is Newport County in RI and belongs in NPCC
#55079 and 55089 are WI counties and belong in RFC
for i in nerc_counties:
    if i[2:9] == [0,0,0,0,0,0,0]:
        if i[0][0] == '0' and i[0][1] == '5':
            i[5] = 1
        if i[0][0] == '1' and i[0][1] == '9':
            i[2] = 1
        if i[0][0] == '2' and i[0][1] == '9':
            i[5] = 1
        if i[0][0] == '4' and i[0][1] == '0':
            i[6] = 1
        if i[0] == '11001':
            i[4] = 1
        if i[0] == '23023':
            i[3] = 1
        if i[0][0] == '2' and i[0][1] == '6':
            i[4] = 1
        if i[0] == '34017':
            i[4] = 1
        if i[0][0] == '3' and i[0][1] == '7':
            i[5] = 1
        if i[0] == '44005':
            i[3] = 1
        if i[0][0] == '5' and i[0][1] == '5':
            i[4] = 1

nerc_counties = pd.DataFrame(nerc_counties)
nerc_counties.columns = ['FIPS', 'County', 'MRO', 'NPCC', 'RFC', 'SERC', 'SPP', 'TRE', 'WECC']

In [49]:
#Create EGU and COUNTY skeletons to ultimately make county x egu array.
#County skeleton array
county_array = np.array(nerc_counties).tolist()
counties_array = np.array(counties).tolist()
county_skel = [[i[4], (i[20]).centroid] for i in counties_array]
for i in range(len(nerc_counties)):
    if county_array[i][2] == 1:
        county_skel[i].insert(2, 1)
    elif county_array[i][3] == 1:
        county_skel[i].insert(2, 2)
    elif county_array[i][4] == 1:
        county_skel[i].insert(2, 3)
    elif county_array[i][5] == 1:
        county_skel[i].insert(2, 4)
    elif county_array[i][6] == 1:
        county_skel[i].insert(2, 5)
    elif county_array[i][7] == 1:
        county_skel[i].insert(2, 6)
    elif county_array[i][8] == 1:
        county_skel[i].insert(2, 7)
    else:
        county_skel[i].insert(2, 'None')


In [57]:
#EGU Skeleton array
egu_array = np.array(nerc_egu).tolist()
egu = np.array(egu).tolist()

egu_skel = [[egu[i][3], Point(float(egu[i][18]),float(egu[i][17]))] for i in range(len(egu))]
for i in range(len(nerc_egu)):
    if egu_array[i][3] == 1:
        egu_skel[i].insert(2, 1)
    elif egu_array[i][4] == 1:
        egu_skel[i].insert(2, 2)
    elif egu_array[i][5] == 1:
        egu_skel[i].insert(2, 3)
    elif egu_array[i][6] == 1:
        egu_skel[i].insert(2, 4)
    elif egu_array[i][7] == 1:
        egu_skel[i].insert(2, 5)
    elif egu_array[i][8] == 1:
        egu_skel[i].insert(2, 6)
    elif egu_array[i][9] == 1:
        egu_skel[i].insert(2, 7)
    else:
        egu_skel[i].insert(2, 'None')

#Delete large datasets to clear up memory
del egu_array

In [59]:
#Create large array that has as many rows as counties and columns as egu's.
#Note that if a county and egu are not in the same NERC region, their intersect will have a '0'.

#create list of shapely points because the points in egu and county skel are of type str and not shapely geometry (for .distance operation)

egu_coords = [Point(float(egu[i][18]),float(egu[i][17])) for i in range(len(egu))]
county_coords = [(i[20]).centroid for i in counties_array]

egu_skel = np.array(egu_skel).tolist()
county_skel = np.array(county_skel).tolist()
distance_weight = []
for i in range(len(county_skel)):
    county_weight = []
    for n in range(len(egu_skel)):
        if len(county_skel[i][2]) <= 2 and str(county_skel[i][2]) == str(egu_skel[n][2]):
            inv_distance = 1/(county_coords[i].distance(egu_coords[n]))
            county_weight.append(inv_distance)
        else:
            county_weight.append(0)
    distance_weight.append(county_weight)

#Delete large datasets to clear up memory
del county_coords
del egu_skel

In [61]:
#Multiply the added MWh per county in the county_vmt file by 1-(renewability fraction) for a county in each nerc reg.

all_county_nerc = []
for i in county_skel:
    if i[2] == 'None':
        pass
    else:
        bucket = []
        bucket.append(int(i[0]))
        bucket.append(int(i[2]))
    all_county_nerc.append(bucket)

for i in county_vmt:
    for n in all_county_nerc:
        if i[0] == n[0]:
            new_mwh = i[-1] * fracs_by_nerc[(n[1])-1]
            i[-1] = new_mwh

del county_skel

In [62]:
#Normalize the weights so that all of the weights for a single county add up to 1.
total_list = []
for i in distance_weight:
    total = sum(i)
    total_list.append(total)
for i in range(len(total_list)):
    for n in range(len(distance_weight[i])):
        if total_list[i] > 0:
            normalized_weight = distance_weight[i][n] / total_list[i]
            distance_weight[i][n] = normalized_weight
        else:
            pass

#Delete large datasets to clear up memory
del total_list

In [65]:
#Now we make the intensity weight! One list with all EGU intensities that will be multiplied over the distance weight.
egu = np.array(egu).tolist()
egu_capfac = [i[24] for i in egu]

#Capfac is a list of percent capacity for all EGU's, if %cap is 1, that means the EGU is out of order, simulated by a 100% capacity and thus cannot generate more elec.
#This will also zero out its value in the distance_weight array when multiplied by 0 and render it as a non-generating unit, as it should be.
capfac = []
for i in egu_capfac:
    if i == 'nan' or i == '0.0000':
        m = 1
        capfac.append(m)
    else:
        floated = float(i)
        capfac.append(floated)

intensity_weight = []
for i in capfac:
    availability = 1 - i
    intensity_weight.append(availability)
egu_ng = []
for i in egu:
    if i[37] == 'nan':
        egu_ng.append(0)
    else:
        egu_ng.append(float(i[37].replace(',', '')))
    
#Multiply by net generation to make room weight. Then multiply by non-baseload factor bc we want to use marginal demand.
room_list = []
for i in range(len(intensity_weight)):
    room = intensity_weight[i] * egu_ng[i] #* egu_nbf[i]
    room_list.append(room)
intensity_weight = room_list

In [66]:
#Now we multiply the capacity factor for each EGU by that EGU's distance weight to create our (almost, not normalized) final weight array!
final_weight = []
for county in distance_weight:
    county_weight = [(county[i] * intensity_weight[i]) for i in range(len(county))]
    final_weight.append(county_weight)

In [68]:
#Create Age weight.

#Then, create list of 7418 numbers per the dataset we are using, each number being the year each EGU started operating. In order, as always. 
#Subtract from 2021 to get age.
year_list = []
for i in tally:
    year_list.append(2021 - i[1])
            
#Divide the weight by the age. Older EGU's get a smaller weight, newer ones a larger weight.
final_weights = []
for i in final_weight:
    list_one = []
    for n in range(len(year_list)):
        weight = (i[n] / year_list[n])
        list_one.append(weight)
    final_weights.append(list_one)
final_weight = final_weights

In [69]:
#Now we normalize the almost final weight array to get our ACTUAL final weight array! Woo!
totals = []
for i in final_weight:
    county_total = sum(i)
    totals.append(county_total)
for i in range(len(totals)):
    for n in range(len(final_weight[i])):
        if totals[i] > 0:
            normalized_weight = final_weight[i][n] / totals[i]
            final_weight[i][n] = normalized_weight
        else:
            pass
        
#Delete large datasets to clear up memory
del distance_weight
del intensity_weight

In [73]:
#Multiply weight factor by county vmt's
counties = np.array(counties)
distributed_energy = []
for i in range(len(counties)):
    fips1 = int(counties[i][4])
    for n in county_vmt:
        fips2 = n[0]
        if fips1 == fips2:
            single_county = [m * n[-1] for m in final_weight[i]]
    distributed_energy.append(single_county)
    
#Delete large datasets to clear up memory
del final_weight
del final_weights
del counties

In [75]:
#Sum all of the columns to get a net added MWh per EGU in the U.S.
added_egu = []
for i in range(len(distributed_energy[0])):
    egu_numbers = []
    for n in range(len(distributed_energy)):
        single_egu = distributed_energy[n][i]
        egu_numbers.append(single_egu)
    added_egu.append(sum(egu_numbers))

#Delete large datasets to clear up memory
del distributed_energy

In [76]:
#First column = added energy, second is total produced by powerplant.
for i in range(len(added_egu)):
    total_mwh = egu[i][37]
    plant_list = [added_egu[i]]
    nox_emis = egu[i][39]
    if total_mwh == 'nan' or total_mwh == '0':
        added_percentage = 0
    else:
        if ',' in total_mwh:
            total_mwh = total_mwh.replace(',','')
        total_mwh = int(total_mwh)
        added_percentage = (added_egu[i] / total_mwh)
    plant_list.append(total_mwh)
    plant_list.append(added_percentage)
    plant_list.append(egu[i][24])
    plant_list.append(str(egu[i][3]))
    plant_list.append(egu[i][22])
    plant_list.append(egu[i][17])
    plant_list.append(egu[i][18])
    added_egu[i] = plant_list

In [78]:
pd.DataFrame(added_egu)

,0,1,2,3,4,5,6,7
0,2.631830e+04,439412,0.059894,0.7270,54429,BIOMASS,31.5825,-87.4889
1,2.010015e+04,46242,0.434673,0.0535,56018,GAS,33.1661,-86.2825
2,1.086699e+06,12770891,0.085092,0.5131,3,COAL,31.0069,-88.0103
3,6.589362e+05,26214623,0.025136,0.8565,46,NUCLEAR,34.7042,-87.1189
4,8.479949e+04,195125,0.434591,0.0298,55409,GAS,33.5883,-85.9731
...,...,...,...,...,...,...,...,...
1905,1.082382e+03,5309,0.203877,0.0152,55477,GAS,44.285,-105.3786
1906,1.617666e+04,710524,0.022767,0.9012,55479,COAL,44.2858,-105.3833
1907,2.750602e+04,734354,0.037456,0.8824,56319,COAL,44.2919,-105.3811
1908,5.918815e+04,821699,0.072031,0.8086,56596,COAL,44.2919,-105.3806


In [82]:
#For LADCO runs:

ptegu_2k18 = pd.read_csv('ptegu_list_2018.csv')
ptegu_2k18 = np.array(ptegu_2k18).tolist()
refinery = []
for i in ptegu_2k18:
    if i[-1] not in refinery:
        refinery.append(i[-1])
        
new_csv = []
for i in added_egu:
    if int(i[4]) in refinery:
        new_csv.append(i)
pd.DataFrame(new_csv)